In [2]:
import os
import pandas as pd
import numpy as np

def get_title_list(path):
    """記事タイトル取得関数"""
    title_list = []
    filenames = os.listdir(path) #ファイル名称一覧取得
    for filename in filenames:
        # 1記事ずつファイルの読み込み
        with open(path+filename, encoding="utf_8_sig") as f:
            title = f.readlines()[2].strip() #各記事テキストの改行2番目に記事タイトルが記載してある
            title_list.append(title)
    return title_list

# データセットの生成(タイトルとラベル付与)
"""
今回は例として与えられた記事タイトルから
どのニュース媒体記事なのか？(独女通信、ITライフハック、MOVIE ENTERの3種類)
を分類する為のデータセットを作成する
"""

#独女通信(ラベル0)
# #ITライフハック(ラベル1)
# #MOVIE ENTER(ラベル2)
title_0 = get_title_list('./text/dokujo-tsushin/')
label_0 = np.full(len(title_0),0)
title_1 = get_title_list('./text/it-life-hack/')
label_1 = np.full(len(title_1),1)
title_2 = get_title_list('./text/movie-enter/')
label_2 = np.full(len(title_2),2)

title_list = title_0 + title_1 + title_2
label_list = np.concatenate([label_0,label_1,label_2])

df = pd.DataFrame({
    'label': label_list,
    'title': title_list
})

# 全データの順番をシャッフル(+index振り直し)
df = df.sample(frac=1 ,random_state=0).reset_index(drop=True)
df.to_csv('livedoor_sentence.csv', sep=',', index=False, encoding='utf_8_sig')

In [5]:
import collections
print(collections.Counter(df['label']))

Counter({1: 871, 2: 871, 0: 871})


In [9]:
from transformers import BertJapaneseTokenizer

"""
使用する前にfugashiとunidic-liteをインストールする。
pip install unidic-lite 
pip install fugashi

・BertJapaneseTokenizerは日本語用のBERTトークナイザ
・from_pretrainedで指定されたPytorchモデルの事前学習を実行する
・"cl-tohoku/bert-base-japanese-v2"は東北大学の日本語事前学習用BERTモデル ※下記補足参照
・do_subword_tokenizeは、サブワードのトークン化をするかどうか
・mecab_kwargsでMeCabユーザー辞書やNeoLogd等の指定も可能
　※"mecab_dic": Noneでデフォルト辞書(UniDic)をOFFにする必要あり
"""
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "cl-tohoku/bert-base-japanese-v2", 
    #do_subword_tokenize=False,
    #mecab_kwargs={"mecab_dic": None, "mecab_option": "-d 'C:\mecab-unidic-neologd'"
)

#適当なキーワードでトークナイズしてみる
text = "楽しくリズム感覚が身につく"

#tokenizer.encodeでテキストをトークンIDに,return_tensors='pt'でPytorch型のテンソル型に変換
ids = tokenizer.encode(text, return_tensors='pt')[0]
wakati = tokenizer.convert_ids_to_tokens(ids) #どのようにトークナイズされたか分かち書きで確認
print(ids)
print(wakati)


tensor([    2, 32589, 17651, 16947,   862,  5128,   893, 12953,     3])
['[CLS]', '楽しく', 'リズム', '感覚', 'が', '身', 'に', 'つく', '[SEP]']
